In [1]:
import pandas as pd

In [86]:
class TextClassifier:
    
    def __init__(self, document, dataset):
        self.document = document
        self.dataset = dataset
        self.similarity_table = []
        
    def classify(self, k, weighted=False):
        #takes the k nearest neighboors
        pass
        
    def create_similarity_table(self):
        pass
    
    def calculate_cosine(self):
        pass

In [87]:
class Dataset:
    
    def __init__(self, data_file, class_file):
        self.df = pd.read_csv(data_file, 
                                    sep=" ", 
                                    skiprows=2, 
                                    names=['doc_id', 'term_id', 'nb_occurences'])
        self.class_df = pd.read_csv(class_file, 
                                      names=['doc_id', 'class'])

In [88]:
class Document:
    
    def __init__(self, doc_id):
        self.doc_id = doc_id
        self.bag_of_words = {}
    
    def create_bag_of_words(self, dataset):
        '''returns a dictionary of all (term_id, occurrences) of the terms present in the document'''
        df = dataset.df.loc[dataset.df['doc_id'] == self.doc_id].reset_index()
        for i in range(df.shape[0]):
            self.bag_of_words[df['term_id'][i]] = df['nb_occurences'][i]
        return self.bag_of_words

In [89]:
my_dataset = Dataset('data/news_articles.mtx', 'data/news_articles.labels')

In [90]:
my_document = Document(1664)

In [85]:
my_document.create_bag_of_words(my_dataset)

my_document.bag_of_words

""

''